## Game timestamp standardization
In this notebook I will try to standardize time in the data set for in game analysis.

In [ ]:
%reload_ext autoreload
%autoreload 2
import os 
import sys
import pandas as pd
import numpy as np
from datetime import datetime
import plotly 
import plotly.graph_objects as go
from datetime import timedelta

sys.path.insert(0, "./../../src/")
from data_loader import load_data

In [ ]:
%%time

# force_recompute flag if we want to completly reload data
seasons = load_data(single_df=True, force_recompute=False)
display(seasons)

In [ ]:
# To get unique elements
# unique_elements = seasons.nunique()
# unique_elements.to_csv("table.csv")

## Load optimization
Not related to this use case, but I need it for wiki so I describe how I optimized space and computation.

In [ ]:
if 2 == 3:
    display(seasons.dtypes)
    for col in seasons:
        print(col, len(seasons[col].unique()))

    categories_arr = ['EVENTMSGACTIONTYPE', 'EVENTMSGTYPE', 'GAME_ID', 'PERIOD', 'PERSON1TYPE',
           'PERSON2TYPE', 'PERSON3TYPE', 'PLAYER1_ID', 'PLAYER1_NAME',
           'PLAYER1_TEAM_ABBREVIATION', 'PLAYER1_TEAM_CITY', 'PLAYER1_TEAM_ID',
           'PLAYER1_TEAM_NICKNAME', 'PLAYER2_ID', 'PLAYER2_NAME',
           'PLAYER2_TEAM_ABBREVIATION', 'PLAYER2_TEAM_CITY', 'PLAYER2_TEAM_ID',
           'PLAYER2_TEAM_NICKNAME', 'PLAYER3_ID', 'PLAYER3_NAME',
           'PLAYER3_TEAM_ABBREVIATION', 'PLAYER3_TEAM_CITY', 'PLAYER3_TEAM_ID',
           'PLAYER3_TEAM_NICKNAME', 'season_name', 'home_shot_distance',
           'visitor_shot_distance']
    seasons[categories_arr] = seasons[categories_arr].astype('category')
    """
    display(seasons)
    with open("tmp_test_category.pkl", "wb") as file:
        pickle.dump(seasons, file)
    """
    display(seasons.dtypes)

## Finding seasons which have corrupt timestamps

In [ ]:
duration_of_each_game = seasons.groupby("GAME_ID").count()["WCTIMESTRING"]	
display(duration_of_each_game)

In [ ]:
correct_time_format_regex = "^\s?([0][0-9]|[1][0-2]|[0-9]):[0-5][0-9]\s?(?:AM|PM|am|pm)$"
correct_t_mask = seasons["WCTIMESTRING"].str.match(correct_time_format_regex)
correct_t_df = seasons[correct_t_mask]
wrong_t_df = seasons[~correct_t_mask]

wrong_data_grouped_game = wrong_t_df.groupby("season_name").count()["GAME_ID"]
correct_data_grouped_game = correct_t_df.groupby("season_name").count()["GAME_ID"]
# replacing string values for plotting purposes
correct_data_grouped_game.index = correct_data_grouped_game.index.str.replace("-", "-20")
wrong_data_grouped_game.index = wrong_data_grouped_game.index.str.replace("-", "-20")

display(wrong_data_grouped_game)
display(correct_data_grouped_game)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=wrong_data_grouped_game.index,
    y=wrong_data_grouped_game.values,
    name="Wrong count"
    )
)
fig.add_trace(go.Scatter(
    x=correct_data_grouped_game.index,
    y=correct_data_grouped_game.values,
    name="Correct count"
    )
)
fig.update_layout(
    title_text="Count of wrong and right WCTIMESTRING values for each season", 
    xaxis_title="Season",
    yaxis_title="Count"
)
fig.show()

## Converting to correct timestamps in datetime

In [ ]:
# filter data to only have seasons after 2012
after_2012_data = correct_t_df[correct_t_df["season_name"] >= "2012-2013"]
# convert the dataframe to correct datetime format
after_2012_data["real_world_time"] = pd.to_datetime(after_2012_data["WCTIMESTRING"], 
                                              format='%I:%M %p').dt.strftime('%H:%M')

# group data by game ID and find start and end of a game
grouped_by_game = after_2012_data.groupby("GAME_ID").agg({"real_world_time": list})
grouped_by_game["game_start_time"] = np.nan
grouped_by_game["game_end_time"] = np.nan

for index, row in grouped_by_game.iterrows():
    # very dumb handling of edge cases and wrong values
    until_midnight = []
    after_midnight = []
    for time_str in row["real_world_time"]:
        if time_str <= "23:59" and time_str >= "10:59":
            until_midnight.append(time_str)
        else:
            after_midnight.append(time_str)
    curr_row_arr = sorted(until_midnight) + sorted(after_midnight)
    start_time = pd.to_datetime(curr_row_arr[0], format='%H:%M')
    end_time = pd.to_datetime(curr_row_arr[-1], format='%H:%M')
    
    if start_time.hour > 10 and end_time.hour < 10:
        # add one day, if clock goes over midnight
        end_time = end_time + timedelta(hours=24)
    grouped_by_game.loc[index, "game_start_time"] = start_time
    grouped_by_game.loc[index, "game_end_time"] = end_time

grouped_by_game["game_duration"] = grouped_by_game["game_end_time"] - \
                                            grouped_by_game["game_start_time"]
# convert to minutes
grouped_by_game["game_duration"] = grouped_by_game["game_duration"].dt.total_seconds().div(60).astype(int)
display(grouped_by_game)

In [ ]:
print("Discarded games", len(grouped_by_game[grouped_by_game["game_duration"] >= 250]))

In [ ]:
# removing all rows which have durations bigger than 250, since most are wrongly formatted
grouped_by_game = grouped_by_game[grouped_by_game["game_duration"] < 250]

## Visualizing durations and removing non-correct values

In [ ]:

# removing date component from date time
grouped_by_game["game_start_t_24h"] = [val.time() for val in grouped_by_game["game_start_time"]]
grouped_by_game["game_end_t_24h"] = [val.time() for val in grouped_by_game["game_end_time"]]
display(grouped_by_game)

In [ ]:
# Visualize average staring and end times of games
# grouped_by_start_t = grouped_by_game.groupby("game_start_time").count()["real_world_time"]
grouped_by_start_t = grouped_by_game.set_index("game_start_time").resample("15min").count()
grouped_by_end_t = grouped_by_game.set_index("game_end_time").resample("15min").count()

fig = go.Figure()
fig.add_trace(go.Bar(
    x=grouped_by_start_t.index.strftime('%H:%M'),
    y=grouped_by_start_t["real_world_time"],
    name="Game started at"
    )
)
fig.add_trace(go.Bar(
    x=grouped_by_end_t.index.strftime('%H:%M'),
    y=grouped_by_end_t["real_world_time"],
    name="Game finished at"
    )
)
fig.update_layout(
    title_text="Games started and finished at time of day", 
    xaxis_title="Hour and minute of day (rounded to 15 minutes)",
    yaxis_title="Count",
)

fig.show()

In [ ]:
# average game start and end
print(grouped_by_end_t.reset_index()["game_end_time"].median())
dates = list(grouped_by_end_t.reset_index().sort_values("game_end_time")["game_end_time"])
print(dates[len(dates)//2])

dates = list(grouped_by_start_t.reset_index().sort_values("game_start_time")["game_start_time"])
print(dates[len(dates)//2])

In [ ]:
# Visualizing durations of games
fig = go.Figure()
fig.add_trace(go.Histogram(
    x=grouped_by_game["game_duration"]
    )
)
fig.update_layout(
    title_text="Histogram of game durations", 
    xaxis_title="Game duration in minutes",
    yaxis_title="Count"
)
fig.update_xaxes(tick0=grouped_by_game["game_duration"].min())
fig.show()

In [ ]:
grouped_by_game["game_duration"].mean()

# Wrongly inputed timestamps handling
TODO, set rows after 250 in duration to correct values

In [ ]:

merged_df_t2 = correct_t_df.merge(wrong_game_durations["real_world_time"], 
                   right_index=True,
                   left_on='GAME_ID')
display(merged_df_t2[merged_df_t2['GAME_ID'] == "0021701054"])

#  & merged_df_t2['WCTIMESTRING'] == '7:30 PM'
# merged_df_t2[(merged_df_t2['GAME_ID'] == "0021700097")]['WCTIMESTRING'].unique()
# display(merged_df_t2[(merged_df_t2['GAME_ID'] == "0021701054")].sort_values("Unnamed: 0"))
merged_df_t2[(merged_df_t2['GAME_ID'] == "0021700097") & \
             (merged_df_t2['WCTIMESTRING'] == '7:30 PM')].iloc[2]